In [ ]:
import sys

In [ ]:
def read_input(file_name):
    with open(file_name, "r") as f:
        # Read number of stellar crystals
        N = int(f.readline().strip())
        stellar_crystals = []
        for _ in range(N):
            x, y, value = map(int, f.readline().strip().split())
            stellar_crystals.append((x, y, value))

        # Read number of void mines
        M = int(f.readline().strip())
        void_mines = []
        for _ in range(M):
            x, y, value = map(int, f.readline().strip().split())
            # Multiply void mine value by -1 as specified
            void_mines.append((x, y, -1 * value))

    return stellar_crystals, void_mines

In [ ]:
def get_polygon_vertices(polygon_edges):
    if not polygon_edges:
        return []
    # Start with the first vertex of the first edge
    vertices = [polygon_edges[0][0]]
    for edge in polygon_edges:
        # Append the ending vertex of each edge (if not already added)
        if edge[1] != vertices[-1]:
            vertices.append(edge[1])
    # Ensure the polygon is closed: last vertex should equal the first
    if vertices[0] != vertices[-1]:
        vertices.append(vertices[0])
    return vertices

In [ ]:
# Block 3: Point-in-polygon check
# First, a helper to check if a point lies exactly on a line segment.
def point_on_line(x, y, x1, y1, x2, y2):
    # For axis-aligned edges, simply check if the point is collinear and within bounds.
    if x1 == x2:  # vertical edge
        if x == x1 and min(y1, y2) <= y <= max(y1, y2):
            return True
    elif y1 == y2:  # horizontal edge
        if y == y1 and min(x1, x2) <= x <= max(x1, x2):
            return True
    return False

In [ ]:
# Next, the robust point-in-polygon function.
def point_in_polygon(x, y, vertices):
    n = len(vertices)
    # First, check if the point lies on any edge (boundary check)
    for i in range(n - 1):  # vertices[-1] is the same as vertices[0]
        x1, y1 = vertices[i]
        x2, y2 = vertices[i + 1]
        if point_on_line(x, y, x1, y1, x2, y2):
            return True

    # Use the ray-casting algorithm for interior points.
    inside = False
    for i in range(n - 1):
        x1, y1 = vertices[i]
        x2, y2 = vertices[i + 1]
        # Check if the horizontal ray from (x, y) intersects with the edge.
        if (y1 > y) != (y2 > y):
            # Compute x-coordinate of the intersection point
            intersect_x = x1 + (y - y1) * (x2 - x1) / (y2 - y1)
            if x < intersect_x:
                inside = not inside
    return inside

In [ ]:
# Block 4: Compute the final percentage.
def compute_final_percentage(stellar_crystals, void_mines, polygon_edges, polygon_valid):
    # If polygon is invalid, set check_value to 0.
    if not polygon_valid:
        check_value = 0
    else:
        # Reconstruct the polygon vertices.
        vertices = get_polygon_vertices(polygon_edges)
        check_value = 0
        # Check each stellar crystal.
        for (x, y, value) in stellar_crystals:
            if point_in_polygon(x, y, vertices):
                check_value += value
        # Check each void mine (its value is negative as read).
        for (x, y, value) in void_mines:
            if point_in_polygon(x, y, vertices):
                check_value += value

    # Total value is the sum of the stellar crystal values.
    total_value = sum(value for (_, _, value) in stellar_crystals)
    # Avoid division by zero.
    percentage = (check_value / total_value * 100) if total_value != 0 else 0
    print("Final verified value is {:.2f} ".format(check_value))
    print("Final Percentage: {:.2f}%".format(percentage))
    print("Total Value Possible: {:.2f}".format(total_value))
    return percentage

###Submission Reading

In [ ]:
def read_submission(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Read net value (cost), number of vertices (V), and number of edges (E)
    net_value = int(lines[0].strip())
    V, E = map(int, lines[1].split(','))

    # Read polygon edges
    edges = []
    for i in range(2, 2 + E):
        line = lines[i].strip()

        # Ensure line follows the format "(x1,y1), (x2,y2)"
        if not (line.startswith('(') and line.endswith(')')):
            raise ValueError(f"Invalid edge format on line {i + 1}: {line}")

        try:
            edge_parts = line.split('), (')  # Splitting at edge separators

            # Removing parentheses
            edge_parts[0] = edge_parts[0].replace('(', '')
            edge_parts[1] = edge_parts[1].replace(')', '')

            # Parsing float values
            x1, y1 = map(float, edge_parts[0].split(', '))
            x2, y2 = map(float, edge_parts[1].split(', '))

            edges.append(((x1, y1), (x2, y2)))

        except (ValueError, IndexError):
            raise ValueError(f"Edge format incorrect on line {i + 1}: {line}")

    return net_value, V, E, edges

In [ ]:
def make_edges_cyclic(edges, epsilon=1e-9):
    def points_equal(p1, p2):
        return (abs(p1[0] - p2[0]) < epsilon and
                abs(p1[1] - p2[1]) < epsilon)

    if not edges:
        return edges.copy()

    remaining_edges = edges.copy()
    ordered_edges = []

    # Start with the first edge
    first_edge = remaining_edges.pop(0)
    ordered_edges.append(first_edge)
    current_end = first_edge[1]

    while remaining_edges:
        found = False
        for i in range(len(remaining_edges)):
            edge = remaining_edges[i]
            if points_equal(edge[0], current_end):
                # Add the edge as is
                ordered_edges.append(edge)
                current_end = edge[1]
                remaining_edges.pop(i)
                found = True
                break
            elif points_equal(edge[1], current_end):
                # Reverse the edge and add
                reversed_edge = (edge[1], edge[0])
                ordered_edges.append(reversed_edge)
                current_end = reversed_edge[1]
                remaining_edges.pop(i)
                found = True
                break
        if not found:
            raise ValueError("Edges do not form a closed polygon")

    # Verify the cycle is closed
    if not points_equal(current_end, ordered_edges[0][0]):
        raise ValueError("Edges do not form a closed cycle")

    return ordered_edges


In [ ]:
def check_max_vertices(V,E,max_vertices=1000):
    return V <= max_vertices and E<=max_vertices and V==E

In [ ]:
def do_edges_intersect(e1, e2):
    (x1, y1), (x2, y2) = e1
    (x3, y3), (x4, y4) = e2

    def on_segment(px, py, qx, qy, rx, ry):
        return min(px, qx) <= rx <= max(px, qx) and min(py, qy) <= ry <= max(py, qy)

    if e1 == e2:
        return False

    if (x1 == x2 and x3 == x4) or (y1 == y2 and y3 == y4):
        return not (max(x1, x3) > min(x2, x4) or max(y1, y3) > min(y2, y4))

    return on_segment(x1, y1, x2, y2, x3, y3) or on_segment(x1, y1, x2, y2, x4, y4)

In [ ]:
def check_non_self_intersecting(edges):
        # Ensure the polygon is closed (first and last edge connect)
    n = len(edges)
    for i in range(n):
        (x1, y1), (x2, y2) = edges[i]
        (nx1, ny1), (nx2, ny2) = edges[(i + 1) % n]  # Next edge (wraps around)
        if (x2, y2) != (nx1, ny1):
            return False  # Adjacent edges must meet at their endpoints

# Check for non-adjacent intersections
    for i in range(n):
        for j in range(i + 2, n - (i == 0)):  # Skip adjacent edges
            if do_edges_intersect(edges[i], edges[j]):
                return False

    return True



    # for i in range(len(edges)):
    #     (x1, y1), (x2, y2) = edges[i]
    #     if i < len(edges) - 1:
    #         (nx1, ny1), (nx2, ny2) = edges[i + 1]
    #         if (x2, y2) != (nx1, ny1):
    #             return False  # Adjacent edges must meet at their endpoints
    #     for j in range(len(edges)):
    #         if abs(i - j) > 1 and do_edges_intersect(edges[i], edges[j]) and abs(i-j)!=(len(edges)-1):
    #             return False
    # return True


In [ ]:
def check_axis_aligned(edges):
    for (x1, y1), (x2, y2) in edges:
        if not (x1 == x2 or y1 == y2):
            return False  # Edge must be either horizontal or vertical
    return True

In [ ]:
def validate_solution(net_value, V, E, edges):
    if not check_max_vertices(V,E):
        print("Invalid: Exceeds maximum allowed vertices (1000) or vertices not equal to number of edges.")
        return False

    if not check_non_self_intersecting(edges):
        print("Invalid: Polygon is self-intersecting.")
        return False

    if not check_axis_aligned(edges):
        print("Invalid: Polygon edges are not axis-aligned.")
        return False

    print("Valid solution.")
    return True

In [ ]:
if __name__ == "__main__":
  # for i in range(10):  # Loop from 0 to 9
  #   input_file = f"input{i:02d}.txt"  # Format the input file name (e.g., input00.txt, input01.txt, etc.)
  #   output_file = f"outputy{i:02d}.txt"  # Format the output file name (e.g., outputy00.txt, outputy01.txt, etc.)

  #   # Read the submission file
  #   net_value, V, E, edges = read_submission(output_file)

  #   # Make edges cyclic
  #   edges = make_edges_cyclic(edges)

  #   # Validate the solution
  #   polygon_valid = validate_solution(net_value, V, E, edges)

  #   # Read the input file
  #   stellar_crystals, void_mines = read_input(input_file)

  #   # Compute the final percentage
  #   compute_final_percentage(stellar_crystals, void_mines, edges, polygon_valid)


    net_value, V, E, edges = read_submission("output00.txt")
    edges = make_edges_cyclic(edges)
    polygon_valid = validate_solution(net_value,V,E,edges)

    polygon_edges = edges
    stellar_crystals, void_mines = read_input("input00.txt")
    compute_final_percentage(stellar_crystals, void_mines, polygon_edges, polygon_valid)

Valid solution.
Final verified value is 3218.00 
Final Percentage: 0.03%
Total Value Possible: 10308572.00
